In [36]:
from scipy.sparse import dok_matrix
import networkx as nx
import numpy as np
import mesa
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [37]:
%load_ext autoreload
%autoreload 2

from trans_infra.trans_infra.simulator import TransInfraNetworkModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import osmnx as ox
def load_network(path) -> nx.Graph:
        """loads in modified OSM graph"""
        G_trans = ox.load_graphml(
                    path,
                    node_dtypes={'index':int, 'x':float, 'y':float, 'general0':float, 'general1':float, 
                                'general2':float, 'general3':float, 'general4':float},    
                    edge_dtypes={'u':int, 'v':int, 'speed':float, 'capacity':float, 'general0':float,
                                'general1':float, 'general2':float, 'general3':float})
        G_trans = G_trans.to_undirected()
        return G_trans

In [39]:
graph = load_network("./osm_dataset/raw/seattle.osm")
model = TransInfraNetworkModel(300, 24, graph)

- bradenton: stuck
- copenhagen: slow
- delft: slow
- west: stuck

In [40]:
from trans_infra.trans_infra.simulator import run_simulation_parallel

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
num_agents = 300
max_episode = 24
num_steps = 24
num_runs = 10

In [48]:
import glob
import time

for osm_path in glob.glob('./osm_dataset/raw/*'):
    print(osm_path)
    start = time.time()
    graph = load_network(osm_path)
    output_df = run_simulation_parallel(num_agents, max_episode, graph, num_runs)
    end = time.time()
    print(output_df)
    print(f'Time: {end-start}\n')

./osm_dataset/raw/east.osm
   run  social_score  times_stuck  times_no_path
0    0      0.068789            0              0
1    1      0.070511            0              0
2    2      0.069511            0              0
3    3      0.068878            0              0
4    4      0.069844            0              0
5    5      0.070278            0              0
6    6      0.070400            0              0
7    7      0.070678            0              0
8    8      0.069389            0              0
9    9      0.071267            0              0
Time: 17.155025959014893

./osm_dataset/raw/copenhagen.osm
   run  social_score  times_stuck  times_no_path
0    0      0.029367           45            523
1    1      0.028211           56            525
2    2      0.032811           43            497
3    3      0.033289           36            504
4    4      0.029044           48            546
5    5      0.035867           34            479
6    6      0.029367           4

10 cores on cpu
32 cores on gpu

per agent steps
1. init
    - precompute home, work, & commute
2. assess what the most needed resource is
    scale_dist_cost = (np.array(dist_cost) / max(dist_cost)) * 8
    urgency = self.model.threshes / (self.curr_resources + scale_dist_cost + 1)

    - computed by threshes / (curr_reses / scaled dists)

### Time
- 1 unit = 30 min
    - 16 sleep, 16 work, 16 social? (or make more)
- change thresh logic to work with time
- reset resources every  ...

### Precompute
- save and load model after precompute for faster init

### Social
- connectivity measure

### Overall
- add traffic
- only search when needed / cut down run time
- *** ^^^ ***